In [1]:
def getKmers(sequence, size, step): 
    output_seq=[]
    for x in range(0, len(sequence) - size, step):
        output_seq.append(sequence[x:x+size])
    return(output_seq)

In [5]:
import pandas as pd
df_input=pd.read_csv("/Users/moradigd/Documents/Plasmid/input_agg.csv")
df_input.columns
df_input=df_input.iloc[:,[0,4]]
df_input

IndexError: positional indexers are out-of-bounds

In [18]:
df_input.columns[0]

'index'

In [ ]:
import matplotlib.pyplot as mp
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
from sklearn import datasets
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

import numpy as np 


label_index=2
for k in range(1,4):
    import pandas as pd
    df_input=pd.read_csv("/Users/moradigd/Documents/Plasmid/input_agg.csv")
    print(df_input.columns)
    df_input=df_input.iloc[:,[0,k]]
    df_input.shape
    df_input=df_input.dropna()
    df_input=df_input.reset_index()
    

     
    for g in range(5,12,3):
        result_f=[]
        print("start")
        print(g)
        unique_sequences_set=[]
#for i in range(len(df_input['seq'])):

        for i in range(df_input.shape[0]):
            #print(i)
            unique_sequences_set=unique_sequences_set+getKmers(df_input['seq'][i], g, 1)
        unique_sequences=list(set(unique_sequences_set))

        print("done")
#"abcdabcva".count("ab")
#df_input['seq'][1].count(unique_sequences[1])

        [df_input['seq'][1].count(i) for i in unique_sequences]
        
        print("start")
        unique_sequences_freq=[]
#for i in range(len(df_input['seq'])):
        for j in range(df_input.shape[0]):
            #print(j)
            unique_sequences_freq.append([df_input['seq'][j].count(i) for i in unique_sequences])
        unique_sequences_freq   
        unique_sequences_freq=pd.DataFrame(unique_sequences_freq)
#unique_sequences_freq.columns=unique_sequences
        unique_sequences_freq
        print("done")


        unique_sequences_freq[['label']]=df_input.iloc[0:df_input.shape[0],label_index]
        unique_sequences_freq
        
        scaler = MinMaxScaler()
        unique_sequences_freq = pd.DataFrame(scaler.fit_transform(unique_sequences_freq), columns=unique_sequences_freq.columns)

        data=unique_sequences_freq.values

        X, y = data[:, :-1], data[:, -1]
        
        
        for m in range(10):
        
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
    
            parameters = {
    #    "loss":["deviance"],
    #"learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    #"min_samples_split": np.linspace(0.1, 0.5, 12),
    #"min_samples_leaf": np.linspace(0.1, 0.5, 12),
        "max_depth":[1,3,5],
    #"max_features":["log2","sqrt"],
    #"criterion": ["friedman_mse",  "mae"],
        "subsample":[ 0.8, 1.0],
        "n_estimators":[5,10,30,100]
            }

            clf = GridSearchCV(GradientBoostingRegressor(), parameters, cv=3, n_jobs=-1)

            clf.fit(X_train, y_train)
            print(clf.score(X_train, y_train))
            print(clf.best_params_)

        #correct_test = correct_data(test)
        #testX = correct_test[predictor].values
            result = clf.predict(X_test)
            
            df_imporatnce=pd.DataFrame({"tag":unique_sequences, "importance":clf.best_estimator_.feature_importances_})
            df_imporatnce=df_imporatnce.loc[df_imporatnce['importance'] > 0]
            df_imporatnce.to_csv("/Users/moradigd/Documents/Plasmid/Kmer_importance_"+str(g)+"_"+str(m)+'_'+str(df_input.columns[2]))
            print(df_imporatnce)

    
            from scipy import stats
            rho, pval = stats.spearmanr(result, y_test)
            print(rho)
            print(pval)
            print(df_input.shape)
            print(df_input.columns)
            result_f.append([rho,pval, g, df_input.columns[2], 'gb',m])
            
            
            #random forests
            parameters = {
    #    "loss":["deviance"],
    #"learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    #"min_samples_split": np.linspace(0.1, 0.5, 12),
    #"min_samples_leaf": np.linspace(0.1, 0.5, 12),
        "min_samples_split":[1,3,5],
    #"max_features":["log2","sqrt"],
    #"criterion": ["friedman_mse",  "mae"],
        #"subsample":[ 0.8, 1.0],
        "n_estimators":[5,10,100]
            }

            clf = GridSearchCV(RandomForestRegressor(), parameters, cv=3, n_jobs=-1)

            clf.fit(X_train, y_train)
            print(clf.score(X_train, y_train))
            print(clf.best_params_)

        #correct_test = correct_data(test)
        #testX = correct_test[predictor].values
            result = clf.predict(X_test)
            
            #df_imporatnce=pd.DataFrame({"tag":Onehot_agg.columns[0:len(Onehot_agg.columns)-1].values, "importance":clf.best_estimator_.feature_importances_})
            #df_imporatnce=df_imporatnce.loc[df_imporatnce['importance'] > 0]
            #df_imporatnce.to_csv("/Users/moradigd/Documents/Plasmid/PB_baps_importance_"+str(g)+"_"+str(m)+'_'+str(df_input.columns[0]))
            #print(df_imporatnce)

    
            from scipy import stats
            rho, pval = stats.spearmanr(result, y_test)
            print(rho)
            print(pval)
            print(df_input.shape)
            print(df_input.columns)
            result_f.append([rho,pval, g, df_input.columns[2], 'rf',m])
        
        
        
            from sklearn.linear_model import LassoCV
        #reg = LassoCV(cv=3, random_state=0).fit(X_train, y_train)
        
            lasso = Lasso(random_state=0, max_iter=10000)
            alphas = np.logspace(-4, -0.5, 10)
    
            tuned_parameters = [{'alpha': alphas}]
            clf = GridSearchCV(lasso, tuned_parameters, cv=3)
            clf.fit(X_train, y_train)
            print(clf.best_params_)
            result = clf.predict(X_test)

    
            from scipy import stats
            rho, pval = stats.spearmanr(result, y_test)
            print(rho)
            print(pval)
            print(df_input.shape)
            print(df_input.columns)
            result_f.append([rho,pval, g, df_input.columns[2], 'lasso',m])
            
        pd.DataFrame(result_f).to_csv("/Users/moradigd/Documents/Plasmid/New_results_screening_agg_scale_"+str(g)+'_'+str(df_input.columns[2])+".csv")
        
        #model = XGBRegressor()
        #model.fit(X[0:int(df_input.shape[0]*0.8) , :] , y[0:int(df_input.shape[0]*0.8) ])
        #model.predict(X[int(df_input.shape[0]*0.8) :df_input.shape[0], :])

        #mp.scatter(model.predict(X[int(df_input.shape[0]*0.8) :df_input.shape[0], :]), y[int(df_input.shape[0]*0.8) :df_input.shape[0]])
        #mp.xlabel('array1')
        #mp.ylabel('array2')
        #from scipy import stats
        #rho, pval = stats.spearmanr(model.predict(X[int(df_input.shape[0]*0.8) :df_input.shape[0], :]), y[int(df_input.shape[0]*0.8) :df_input.shape[0]])
        #print(rho)
        #print(pval)
        #print(df_input.shape)
        #print(df_input.columns)
        #result.append([rho,pval, g, df_input.columns[2]])


Index(['seq', 'PKJK', 'RP', 'PB'], dtype='object')
start
5


In [8]:
data.shape

(589, 1025)

In [11]:
unique_sequences_freq.to_csv("/Users/moradigd/Documents/Plasmid/interval.csv")

In [10]:
unique_sequences_freq

,0,1,2,3,4,5,6,7,8,9,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,label
0,0,0,0,0,0,1,0,0,0,2,...,0,0,0,1,1,0,0,1,1,8.144022
1,0,0,2,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,2,3.840624
2,0,1,1,0,0,2,0,1,0,1,...,0,0,1,1,0,0,2,0,0,0.000000
3,0,0,0,0,0,0,0,0,0,2,...,0,0,0,1,0,1,2,1,0,7.132588
4,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,23.074729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,1,0,2,1,0,2,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0.000000
585,1,0,2,0,0,2,0,0,0,2,...,0,0,0,0,0,0,1,1,0,4.072011
586,0,0,1,1,1,2,0,0,0,0,...,0,0,0,0,0,0,2,1,0,18.654461
587,0,0,1,1,1,2,0,0,0,0,...,0,0,0,0,0,0,2,1,1,8.778570


In [27]:
#Subset
import matplotlib.pyplot as mp
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
from sklearn import datasets
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

import numpy as np 

subsize=np.arange(0.1,1,0.1)
label_index=2
for k in range(1,4):
    import pandas as pd
    df_input=pd.read_csv("/Users/moradigd/Documents/Plasmid/input_agg.csv")
    print(df_input.columns)
    df_input=df_input.iloc[:,[0,k]]
    df_input.shape
    df_input=df_input.dropna()
    df_input=df_input.reset_index()
    

    for g in range(5,9,3):
        result_f=[]
        print("start")
        print(g)
        unique_sequences_set=[]

        for i in range(df_input.shape[0]):
            #print(i)
            unique_sequences_set=unique_sequences_set+getKmers(df_input['seq'][i], g, 1)
        unique_sequences=list(set(unique_sequences_set))

        print("done")


        [df_input['seq'][1].count(i) for i in unique_sequences]
        
        print("start")
        unique_sequences_freq=[]
        for j in range(df_input.shape[0]):
            #print(j)
            unique_sequences_freq.append([df_input['seq'][j].count(i) for i in unique_sequences])
        unique_sequences_freq   
        unique_sequences_freq=pd.DataFrame(unique_sequences_freq)
        unique_sequences_freq
        print("done")


        unique_sequences_freq[['label']]=df_input.iloc[0:df_input.shape[0],label_index]
        unique_sequences_freq
        
        scaler = MinMaxScaler()
        unique_sequences_freq = pd.DataFrame(scaler.fit_transform(unique_sequences_freq), columns=unique_sequences_freq.columns)

        data=unique_sequences_freq.values

        X, y = data[:, :-1], data[:, -1]
        
        
        for m in range(5):
        
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
            
            for k in range(len(subsize)):
            
                selected_ids=np.random.choice(X_train.shape[0], int(subsize[k]*X_train.shape[0]) , replace=False)
                X_train_tmp=X_train[selected_ids]
                y_train_tmp=y_train[selected_ids]
    
            #random forests
                parameters = {
    #    "loss":["deviance"],
    #"learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    #"min_samples_split": np.linspace(0.1, 0.5, 12),
    #"min_samples_leaf": np.linspace(0.1, 0.5, 12),
            "min_samples_split":[1,3,5],
    #"max_features":["log2","sqrt"],
    #"criterion": ["friedman_mse",  "mae"],
        #"subsample":[ 0.8, 1.0],
            "n_estimators":[5,10,100]
                }

                clf = GridSearchCV(RandomForestRegressor(), parameters, cv=3, n_jobs=-1)

                clf.fit(X_train_tmp, y_train_tmp)
                print(clf.score(X_train_tmp, y_train_tmp))
                print(clf.best_params_)

        #correct_test = correct_data(test)
        #testX = correct_test[predictor].values
                result = clf.predict(X_test)
            
            #df_imporatnce=pd.DataFrame({"tag":Onehot_agg.columns[0:len(Onehot_agg.columns)-1].values, "importance":clf.best_estimator_.feature_importances_})
            #df_imporatnce=df_imporatnce.loc[df_imporatnce['importance'] > 0]
            #df_imporatnce.to_csv("/Users/moradigd/Documents/Plasmid/PB_baps_importance_"+str(g)+"_"+str(m)+'_'+str(df_input.columns[0]))
            #print(df_imporatnce)

    
                from scipy import stats
                rho, pval = stats.spearmanr(result, y_test)
                print(rho)
                print(pval)
                print(X_train_tmp.shape)
                print(df_input.columns)
                result_f.append([rho,pval, g, df_input.columns[2], 'rf',m,subsize[k], "test"])
                
                result = clf.predict(X_train_tmp)
                rho, pval = stats.spearmanr(result, y_train_tmp)
                print(rho)
                print(pval)
                print(X_train_tmp.shape)
                print(df_input.columns)
                result_f.append([rho,pval, g, df_input.columns[2], 'rf',m,subsize[k], "train"])

        pd.DataFrame(result_f).to_csv("/Users/moradigd/Documents/Plasmid/Subset_New_results_screening_agg_scale_"+str(g)+'_'+str(df_input.columns[2])+"_"+str(subsize[k])+".csv")
        

Index(['seq', 'PKJK', 'RP', 'PB'], dtype='object')
start
5
done
start
done


/Users/moradigd/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [        nan         nan         nan -0.54298775 -0.2504339  -0.26537772
 -0.46195724 -0.27409332 -0.30034439]
  warnings.warn(


0.8166576101426506
{'min_samples_split': 3, 'n_estimators': 10}
0.2964288272003267
6.04437203174437e-10
(502, 1024)
Index(['index', 'seq', 'PKJK'], dtype='object')
0.8182730477311222
2.561679878486327e-122
(502, 1024)
Index(['index', 'seq', 'PKJK'], dtype='object')


/Users/moradigd/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [        nan         nan         nan -0.2708128  -0.20256176 -0.08963157
 -0.25140455 -0.18008642 -0.07089354]
  warnings.warn(


0.7413275220362748
{'min_samples_split': 5, 'n_estimators': 100}
0.35335812174493236
9.106988162359656e-14
(669, 1024)
Index(['index', 'seq', 'PKJK'], dtype='object')
0.8621999186156819
3.526584947559321e-199
(669, 1024)
Index(['index', 'seq', 'PKJK'], dtype='object')


KeyboardInterrupt: 

In [12]:
subsize=np.arange(0.2,1,0.2)

In [3]:
import numpy as np

In [47]:
df_input.take([i]).values[0]

'GGCAGGTATGTTACCACACCGATTTCACATAGATTGCTGGCTGATTGCATA'

In [48]:

type(df_input)


unique_sequences_set=[]
#for i in range(len(df_input['seq'])):

for i in range(df_input.shape[0]):
    #print(i)
    print(df_input.take([i]).values[0])
    unique_sequences_set=unique_sequences_set+getKmers(df_input.take([i]).values[0], 5, 1)
unique_sequences=list(set(unique_sequences_set))
unique_sequences_freq

GGCAGGTATGTTACCACACCGTTTTCACATAGATTGCTGGCTGATAGCATA
GGCAGGTATGTTACCACACCGTTTTCACATAGATTGCTGGCTGATAGCATA
TGCAGCTATGTTACCACACCGTTTTCACATAGATTGCTGGCTGATAGCATA
TGCAGCTATGTTACCACACCGTTTTCACATAGATTGCTGGCTGATAGCATA
TGCAGCTATGTTACCACACCGTTTTCACATAGATTGCTGGCTGATAGCATA
GGCAGGTATGTTACCATCCCGTTTTCACCTAGATTGCTGGCTGATAGCATA
TGCAGCTATGTTACCACACCGTTTTCACATAGATTGCTAGCTGATAGCATA
GGTTCGGATATGTCCACACCGTTTTCACATAGATTGCTGGCTGATAGCATA
GGCAGGTATGTTACCACACCGTTTTCACATAGAGTGATGGCTGATAGCATA
GGTTCGGATATGTCCACACCGTTTTCACATAGATGGCTGGCTGATACCATA
GGCAGGTATGTTACCACACCGTTTTCACATAGATTGCTGGCTGATAGCATA
TGCAGCTATGTTACCACACCGTTTTCACATAGATTGCTGGCTGATAGCATA
GGCAGGTATGTTACCACACCGTTTTCACATAGATTGCTGGCTGATAGCATA
GGCAGGTATGTTACCACACCGTTTTCACATAGATTGCTGGCTGATAGCATA
GGCAGGTATGTTACCACACCGTTTTCACATAGAGTGCTGGCTGATAGCATA
TGCAGCTATGTTACCACACCGTTTTCACATAGATTGCTGGCTGATAGCATA
GGTTCGGATATGTCCACACCGTTTTCACATAGATTGCTGGCTGATAGCATA
GGCAGGTATGTTACCACCCCGTTTTCACCTAGATTGCTGGCTGATAGCATA
GGCAGGTATGTTACCACACCGTTTTCACATAGATTGCTGGCTGATAGCATA
GGTTCGGATATG

['AAGCA',
 'TCCCA',
 'AGAGT',
 'AAGAT',
 'AAATA',
 'AAAGA',
 'ACACA',
 'CTGAT',
 'GGGAA',
 'GGTAT',
 'CCAAA',
 'TATGT',
 'CCTAG',
 'TACAT',
 'ATCGA',
 'CACAA',
 'TGAGC',
 'CTTTC',
 'ATTAC',
 'GGGTG',
 'TCCCG',
 'ACGGA',
 'ACGTT',
 'GCTAG',
 'GATTT',
 'GATGT',
 'TTGCT',
 'TGATA',
 'TCCAC',
 'GATAG',
 'TTCAC',
 'CTAGC',
 'TGATT',
 'GTTTA',
 'ATACA',
 'ATAAA',
 'AGTGA',
 'GCAGG',
 'TTTTC',
 'GATGG',
 'GGAAA',
 'TACCT',
 'GCACA',
 'GTACG',
 'AAGAG',
 'CTATG',
 'CATCG',
 'CCACA',
 'GGTTC',
 'ACAAC',
 'GGGGG',
 'TTTCC',
 'GTTCG',
 'TGACC',
 'GAGTG',
 'TGTTA',
 'AGATG',
 'CTGTT',
 'AACTT',
 'CGGGT',
 'AGGTA',
 'GGCTA',
 'ACTGT',
 'ATGGC',
 'TAATA',
 'AGACT',
 'GCTGA',
 'CGATT',
 'TGCTG',
 'TTGCA',
 'CGTTT',
 'CAGCT',
 'ACAAA',
 'TCGGG',
 'GTATG',
 'AGCAT',
 'TTTAA',
 'GCATA',
 'CAACG',
 'ACCCT',
 'TTCTG',
 'GGATA',
 'ACCAA',
 'CTACA',
 'ATGGA',
 'AGCTA',
 'CCACC',
 'TTCGG',
 'TACGT',
 'AGCTG',
 'ATATG',
 'GATAT',
 'CCCCG',
 'ACCTC',
 'CGGAT',
 'ACCCC',
 'ACCGA',
 'GTGGA',
 'GTGGG',
 'TTCCC',


In [62]:
#selected_ids=np.random.choice(X_train.shape[0], int(subsize[0]*X_train.shape[0]) , replace=False)
unique_sequences_freq

,0,1,2,3,4,5,6,7,8,9,...,224,225,226,227,228,229,230,231,232,233
0,0,0,0,0,0,0,0,1,0,1,...,0,0,0,1,0,0,0,0,0,1
1,0,0,0,0,0,0,0,1,0,1,...,0,0,0,1,0,0,0,0,0,1
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,1,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
95,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
96,0,0,0,0,0,0,0,1,0,1,...,0,0,0,1,0,0,0,0,0,1
97,1,0,0,0,1,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,0


In [36]:
import pandas as pd
df_input=pd.read_csv("/Users/moradigd/Documents/Plasmid/SimBac-master/variant_genomes_0.01_100.vcf")
df_input




In [2]:
import pandas as pd
df_input_original=pd.read_csv("/Users/moradigd/Documents/Plasmid/SimBac-master/variant_genomes_0.01_100.vcf")
df_input_original.columns
sequence_of_numbers = [number+1 for number in range(0, df_input_original.shape[0]) if (number % 2 in (1, 2))]
df_input=df_input_original.iloc[sequence_of_numbers,0]

for g in range(5,6):
        result_f=[]
        print("start")
        print(g)
        unique_sequences_set=[]
#for i in range(len(df_input['seq'])):

        for i in range(df_input.shape[0]):
            #print(i)
            unique_sequences_set=unique_sequences_set+getKmers(df_input.take([i]).values[0], 5, 1)
            #unique_sequences_set=unique_sequences_set+getKmers(df_input.take([i]), g, 1)
        unique_sequences=list(set(unique_sequences_set))

        print("done")

        
#"abcdabcva".count("ab")
#df_input['seq'][1].count(unique_sequences[1])

        #[df_input['seq'][1].count(i) for i in unique_sequences]
        
        print("start")
        unique_sequences_freq=[]
#for i in range(len(df_input['seq'])):
        for j in range(df_input.shape[0]):
            #print(j)
            unique_sequences_freq.append([df_input.take([j]).values[0].count(i) for i in unique_sequences])
        unique_sequences_freq
        unique_sequences_freq=pd.DataFrame(unique_sequences_freq)
#unique_sequences_freq.columns=unique_sequences
        unique_sequences_freq
        print("done")


        #unique_sequences_freq[['label']]=df_input.iloc[0:df_input.shape[0],df_input_original.iloc[[i-1 for i in sequence_of_numbers],0].values]
        unique_sequences_freq

start
5
done
start
done


In [3]:
unique_sequences_freq.to_csv("")
#[i+1 for i in sequence_of_numbers]

,0,1,2,3,4,5,6,7,8,9,...,224,225,226,227,228,229,230,231,232,233
0,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,1
1,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,1
2,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,0,0,1,0,0,0,0,0,0,0,...,0,1,1,0,0,1,0,0,0,0
95,0,1,1,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
96,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,1
97,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,1,0,0,1,0


In [12]:
#simulation 







start
5


KeyError: 'seq'